# AIA Malaysia Funds Analysis

@author : 钟智强

> This application is designed to retrieve and analyze the most up-to-date fund details, enabling comprehensive comparisons for informed decision-making.

1. Download The Dataset from AIA Website

In [4]:
# Import Necessary Packages
try:
    import os
    import time
    import platform
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.chrome.options import Options
    from webdriver_manager.chrome import ChromeDriverManager
except ImportError as e:
    display(f"Failed to import necessary packages: {e}")
else:
    display("All packages imported successfully")

'All packages imported successfully'

In [5]:
# Base Url Declaration

_AIA_FUND_ = "https://www.aia.com.my/en/our-products/investment-linked-funds.html"

In [6]:
global driver

if not os.path.isfile("../data/Funds_List.csv"):
    if platform.system() != "Darwin":
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Run Chrome in headless mode

        # Initialize WebDriver with ChromeDriverManager
        # Download from https://chromedriver.storage.googleapis.com/index.html?path=114.0.5735.90/
        # or
        # https://commondatastorage.googleapis.com/chromium-browser-snapshots/index.html?prefix=Mac_Arm/1280672/
        service = Service("config/chromedriver")
        # Uncomment this line to enable chrome
        # driver = webdriver.Chrome(
        #     service=service, options=chrome_options
        # )
        driver = None # Comment tis line if using chrome driver
    else:
        # Initialize WebDriver for Safari
        driver = webdriver.Safari()

    driver.get(_AIA_FUND_)

    display(driver)

    download_button = driver.find_element(
        "xpath",
        '//button[@type="submit" and contains(@class, "cmp-x3__button") and .//span[contains(text(), "Export list to CSV file")]]',
    )
    download_button.click()

    driver.quit()
else:
    pass

2. Load and Explore the Data
> First, load the CSV file into a pandas DataFrame and explore the data. This allows us to get an overview and perform initial filtering.

In [7]:
_AIA_FUND_LIST_ = "data/Funds_List.csv"

# Load the CSV file
df = pd.read_csv(_AIA_FUND_LIST_)

# Display the first few rows of the DataFrame
display(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'data/Funds_List.csv'

3. Filter and Identify the Best Funds
> We can filter the data based on various criteria, such as the risk level, asset allocation, or performance (e.g., NAV price, movement). For illustration, let's assume we want to find the top 5 funds based on the NAV Price or Movement.

In [ ]:
# Convert 'Nav Price' and 'Movement' to numeric, if they are not already
df["Nav Price"] = pd.to_numeric(df["Nav Price"], errors="coerce")
df["Movement"] = pd.to_numeric(df["Movement"], errors="coerce")

# Sort by NAV Price or Movement to find the best-performing funds
top_funds_by_nav = df.sort_values(by="Nav Price", ascending=False).head(5)
top_funds_by_movement = df.sort_values(by="Movement", ascending=False).head(5)

display("Top Funds by NAV Price:")
display(top_funds_by_nav)

display("Top Funds by Movement:")
display(top_funds_by_movement)

: 

4. Visualize the Data

-  4a. Bar Chart for NAV Price and Movement

In [ ]:
# Set up the plotting style
sns.set(style="whitegrid")

# Create a bar plot for NAV Price
plt.figure(figsize=(12, 6))
nav_plot = sns.barplot(
    x="Nav Price",
    y="Fund Name",
    data=top_funds_by_nav,
    palette="viridis",
    legend=False,
    hue="Nav Price",
)
plt.title("Top Funds by NAV Price", fontsize=16, fontweight="bold")
plt.xlabel("NAV Price", fontsize=14)
plt.ylabel("Fund Name", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add data labels
for p in nav_plot.patches:
    nav_plot.annotate(
        format(p.get_width(), ".2f"),
        (p.get_width(), p.get_y() + p.get_height() / 2),
        ha="center",
        va="center",
        xytext=(10, 0),
        textcoords="offset points",
        fontsize=12,
        color="black",
    )

plt.show()

# Create a bar plot for Movement
plt.figure(figsize=(12, 6))
movement_plot = sns.barplot(
    x="Movement", y="Fund Name", data=top_funds_by_movement, palette="viridis",legend=False, hue="Movement"
)
plt.title("Top Funds by Movement", fontsize=16, fontweight="bold")
plt.xlabel("Movement", fontsize=14)
plt.ylabel("Fund Name", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add data labels
for p in movement_plot.patches:
    movement_plot.annotate(
        format(p.get_width(), ".2f"),
        (p.get_width(), p.get_y() + p.get_height() / 2),
        ha="center",
        va="center",
        xytext=(10, 0),
        textcoords="offset points",
        fontsize=12,
        color="black",
    )

plt.show()

: 

- 4b. Scatter Plot to Compare NAV Price and Movement

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=df, x="Nav Price", y="Movement", hue="Risk Level", palette="Set1", s=100
)
plt.title("NAV Price vs. Movement by Risk Level")
plt.xlabel("NAV Price")
plt.ylabel("Movement")
plt.legend(title="Risk Level")
plt.show()

: 

5. Additional Filtering and Comparison
> We might want to filter funds based on asset allocation, market, or Shariah compliance. 

In [ ]:
# Filter funds that are Shariah compliant
shariah_funds = df[df["Shariah/Non-Sariah"] == "Shariah"]

# Filter by Asset Allocation (e.g., 'Local')
local_funds = df[df["Asset Allocation"] == "Local"]

# Display the filtered DataFrames
display("Shariah Compliant Funds:")
display(shariah_funds)

: 

6. Get the Fund with the Highest Return
We can interpret "highest return" as the highest NAV Price or Movement.

In [ ]:
# Get the fund with the highest NAV Price
highest_nav_fund = df.loc[df["Nav Price"].idxmax()]

# Display the fund with the highest NAV Price
display("Fund with the Highest NAV Price:")
display(highest_nav_fund)

: 

7. Visualize the Fund with the Highest Return

In [ ]:
# Set up the plotting style
sns.set(style="whitegrid")

# Highlight the highest NAV fund in the bar chart
plt.figure(figsize=(12, 6))
nav_plot = sns.barplot(
    x="Nav Price",
    y="Fund Name",
    data=top_funds_by_nav,
    palette="viridis",
    legend=False,
    hue="Fund Name",
)
plt.title("Top Funds by NAV Price", fontsize=16, fontweight="bold")
plt.xlabel("NAV Price", fontsize=14)
plt.ylabel("Fund Name", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Highlight the highest NAV fund
highest_nav_name = highest_nav_fund["Fund Name"]
for p in nav_plot.patches:
    if (
        p.get_y()
        == top_funds_by_nav[top_funds_by_nav["Fund Name"] == highest_nav_name].index[0]
    ):
        nav_plot.annotate(
            format(p.get_width(), ".2f"),
            (p.get_width(), p.get_y() + p.get_height() / 2),
            ha="center",
            va="center",
            xytext=(10, 0),
            textcoords="offset points",
            fontsize=12,
            color="red",  # Highlight in red
            weight="bold",
        )
    else:
        nav_plot.annotate(
            format(p.get_width(), ".2f"),
            (p.get_width(), p.get_y() + p.get_height() / 2),
            ha="center",
            va="center",
            xytext=(10, 0),
            textcoords="offset points",
            fontsize=12,
            color="black",
        )

plt.show()

: 

8. Visualize Risk Levels


In [ ]:
# Violin plot for NAV Price by Risk Level
plt.figure(figsize=(12, 6))
sns.violinplot(x="Risk Level", y="Nav Price", data=df, palette="Set1")
plt.title("NAV Price Distribution by Risk Level", fontsize=16, fontweight="bold")
plt.xlabel("Risk Level", fontsize=14)
plt.ylabel("NAV Price", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Violin plot for Movement by Risk Level
plt.figure(figsize=(12, 6))
sns.violinplot(x="Risk Level", y="Movement", data=df, palette="Set1")
plt.title("Movement Distribution by Risk Level", fontsize=16, fontweight="bold")
plt.xlabel("Risk Level", fontsize=14)
plt.ylabel("Movement", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

: 

In [ ]:
# Bar plot of NAV Price by Risk Level
plt.figure(figsize=(12, 6))
sns.barplot(x="Risk Level", y="Nav Price", data=df, palette="Set1", errorbar=None, hue="Risk Level")
plt.title("NAV Price by Risk Level", fontsize=16, fontweight="bold")
plt.xlabel("Risk Level", fontsize=14)
plt.ylabel("NAV Price", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Bar plot of Movement by Risk Level
plt.figure(figsize=(12, 6))
sns.barplot(
    x="Risk Level",
    y="Movement",
    data=df,
    palette="Set1",
    errorbar=None,
    hue="Risk Level",
)
plt.title("Movement by Risk Level", fontsize=16, fontweight="bold")
plt.xlabel("Risk Level", fontsize=14)
plt.ylabel("Movement", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

: 